In [20]:
import numpy as np
import pandas as pd

In [21]:
import sqlalchemy
#connect to mysql
database_username = 'namnp'
database_password = '12345678'
database_ip       = '127.0.0.1'
database_name     = 'iot'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [22]:
df = pd.read_sql('SELECT * FROM `traffic_with_lag_data` where REPORT_ID = 158324 order by REPORT_ID, timestamp ', con=database_connection)
df.head(3)

,time,REPORT_ID,DISTANCE_IN_METERS,vehicleCount,timestamp,day_of_week,vehicleCount_lag_1,vehicleCount_lag_2
0,9,158324,1030,75,2014-08-01 09:00:00,4,78.0,11.0
1,10,158324,1030,50,2014-08-01 10:00:00,4,75.0,78.0
2,11,158324,1030,100,2014-08-01 11:00:00,4,50.0,75.0


In [23]:
cols = ['time','day_of_week','REPORT_ID','DISTANCE_IN_METERS','vehicleCount_lag_1','vehicleCount_lag_2','vehicleCount']
df = df[cols]
df.head(3)

,time,day_of_week,REPORT_ID,DISTANCE_IN_METERS,vehicleCount_lag_1,vehicleCount_lag_2,vehicleCount
0,9,4,158324,1030,78.0,11.0,75
1,10,4,158324,1030,75.0,78.0,50
2,11,4,158324,1030,50.0,75.0,100


In [24]:
def loss(h, y):
  sq_error = (h - y)**2
  n = len(y)
  return 1.0 / (2*n) * sq_error.sum()

In [25]:
class LinearRegression:
  
  def predict(self, X):
    return np.dot(X, self._W)
  
  def _gradient_descent_step(self, X, targets, lr):

    predictions = self.predict(X)
    
    error = predictions - targets
    gradient = np.dot(X.T,  error) / len(X)

    self._W -= lr * gradient
      
  def fit(self, X, y, n_iter=100000, lr=0.01):

    self._W = np.zeros(X.shape[1])

    self._cost_history = []
    self._w_history = [self._W]
    for i in range(n_iter):
      
        prediction = self.predict(X)
        cost = loss(prediction, y)
        
        self._cost_history.append(cost)
        
        self._gradient_descent_step(x, y, lr)
        
        self._w_history.append(self._W.copy())
    return self

In [26]:
x = df[['time','day_of_week','REPORT_ID','DISTANCE_IN_METERS','vehicleCount_lag_1','vehicleCount_lag_2']]

x = (x - x.mean()) / x.std()
x = np.c_[np.ones(x.shape[0]), x] 

In [27]:
x.shape

(1328, 7)

In [28]:
y = df['vehicleCount']

In [29]:
clf = LinearRegression()
clf.fit(x, y, n_iter=2000, lr=0.01)

In [30]:
result = clf.predict(x)

In [31]:
result

array([nan, nan, nan, ..., nan, nan, nan])